# Test download da ftp

!!! la separazione dei campi non è del tutto corretta


In [33]:
from ftplib import FTP
import pandas as pd
from io import StringIO

In [34]:
def download_file_list_from_ftp(server, port, user, password):
    ftp = FTP()
    ftp.connect(server, port)
    ftp.login(user, password)

    # Get the list of files in the FTP directory
    file_list = []
    ftp.retrlines('NLST', file_list.append)

    ftp.quit()

    return file_list

In [35]:
def download_data_from_ftp(server, port, user, password, filename):
    ftp = FTP()
    ftp.connect(server, port)
    ftp.login(user, password)

    # Download the data file
    with open(filename, 'wb') as file:
        ftp.retrbinary('RETR ' + filename, file.write)

    ftp.quit()

In [36]:
def read_data_file(filename, decimal_separator, field_positions):
    # Read the data file and parse it into a dataframe
    with open(filename, 'r') as file:
        # Skip rows 1 to 9
        for _ in range(9):
            next(file)

        # Read row 10 as headers
        headers_line = next(file)
        headers = [header.strip() for header in headers_line.split()]

        # Skip row 11
        next(file)

        # Read the remaining data
        data = []
        for line in file:
            row = []
            for start, end in field_positions:
                field = line[start-1:end].strip().replace(decimal_separator, '.')
                row.append(field)
            data.append(row)

    # Create a dataframe
    df = pd.DataFrame(data, columns=headers)

    return df

In [37]:
def download_and_combine_data(server, port, user, password, file_list, decimal_separator, field_positions):
    all_data = []

    for filename in file_list:
        print(f"Downloading {filename}...")
        download_data_from_ftp(server, port, user, password, filename)

        # Read data from the downloaded file and append it to the list
        df = read_data_file(filename, decimal_separator, field_positions)
        all_data.append(df)

    # Concatenate all dataframes into a single dataframe
    combined_dataframe = pd.concat(all_data, ignore_index=True)

    return combined_dataframe


In [38]:

# FTP server credentials and file information
ftp_server = "10.30.27.160"
ftp_port = 21
ftp_user = "meteobridgeftp"
decimal_separator = "."
ftp_password = input()

# Define field positions as (start_position, end_position)
field_positions = [
    (1, 3),   # DAY
    (6, 9),   # TEMP
    (12, 15),  # HIGH
    (18, 23),  # TIME
    (26, 29),  # LOW
    (32, 37),  # TIME
    (40, 44),  # RAIN
    (47, 51),  # WIND
    (54, 58),  # HIGH
    (61, 66),  # TIME
    (68, 71),  # DIR
    (73, 79),  # BAROM
    (82, 84)   # HUM
]

In [39]:

# Download list of files
file_list = download_file_list_from_ftp(ftp_server, ftp_port, ftp_user, ftp_password)
print(file_list)


# Download and combine data from all files
combined_dataframe = download_and_combine_data(ftp_server, ftp_port, ftp_user, ftp_password, file_list, decimal_separator, field_positions)




error_perm: 530 Login incorrect.

In [ ]:
# Display the combined dataframe
combined_dataframe.to_csv('test.csv', sep = "|", index = False)
combined_dataframe

,DAY,TEMP,HIGH,TIME,LOW,TIME,RAIN,WIND,HIGH,TIME,DIR,BAROM,HUM
0,19,-0.,-0,00:0,-0.,00:01,0.0,2.1,3.2,00:03,305,1033.6,7
1,19,-0.,-0,00:0,-0.,00:07,0.0,2.3,3.2,00:08,305,1033.6,7
2,19,-0.,-0,00:1,-0.,00:13,0.0,1.8,3.2,00:11,305,1033.6,7
3,19,-0.,-0,00:1,-0.,00:17,0.0,2.0,3.2,00:18,305,1033.6,6
4,19,-0.,-0,00:2,-0.,00:23,0.0,2.5,3.2,00:23,305,1033.5,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
868,20,6.,7,23:5,6.,23:53,0.0,7.7,15.5,23:52,174,1016.0,5
869,20,6.,7,:,6.,:,0.0,7.7,15.5,:,174,1016.0,5
870,---,----,----,------,----,------,-----,-----,-----,------,----,-------,-
871,MON,5.,17,19,-3.,04,111.2,5.5,68.4,02,SW,1020.4,0
